<a href="https://colab.research.google.com/github/fumio125/machikane_cv/blob/main/keypoint.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [55]:
from IPython.display import display, Javascript, Image
from google.colab.output import eval_js
from google.colab.patches import cv2_imshow
from base64 import b64decode, b64encode
import cv2
import numpy as np
import PIL
import io
import html
import time
import copy
import itertools

# Googleドライブへのマウント
from google.colab import drive
drive.mount('/content/drive')
%cd "/content/drive/My Drive/Colab Notebooks/lecture/"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/Colab Notebooks/lecture


In [59]:
# カメラ接続用プログラム

def js_to_image(js_reply):
  image_bytes = b64decode(js_reply.split(',')[1])
  jpg_as_np = np.frombuffer(image_bytes, dtype=np.uint8)
  img = cv2.imdecode(jpg_as_np, flags=1)
  return img

def img_to_bytes(img_array):
  img_PIL = PIL.Image.fromarray(img_array, 'RGBA')
  iobuf = io.BytesIO()
  img_PIL.save(iobuf, format='png')
  img_bytes = 'data:image/png;base64,{}'.format((str(b64encode(iobuf.getvalue()), 'utf-8')))
  return img_bytes

def video_stream():
  js = Javascript('''
    var video;
    var div = null;
    var stream;
    var captureCanvas;
    var imgElement;
    var labelElement;
    
    var pendingResolve = null;
    var shutdown = false;
    
    function removeDom() {
       stream.getVideoTracks()[0].stop();
       video.remove();
       div.remove();
       video = null;
       div = null;
       stream = null;
       imgElement = null;
       captureCanvas = null;
       labelElement = null;
    }
    
    function onAnimationFrame() {
      if (!shutdown) {
        window.requestAnimationFrame(onAnimationFrame);
      }
      if (pendingResolve) {
        var result = "";
        if (!shutdown) {
          captureCanvas.getContext('2d').drawImage(video, 0, 0, 1280, 480);
          result = captureCanvas.toDataURL('image/jpeg', 0.5)
        }
        var lp = pendingResolve;
        pendingResolve = null;
        lp(result);
      }
    }
    
    async function createDom() {
      if (div !== null) {
        return stream;
      }

      div = document.createElement('div');
      div.style.border = '2px solid black';
      div.style.padding = '3px';
      div.style.width = '100%';
      div.style.maxWidth = '800px';
      document.body.appendChild(div);
      
      const modelOut = document.createElement('div');
      modelOut.innerHTML = "<span>Status:</span>";
      labelElement = document.createElement('span');
      labelElement.innerText = 'No data';
      labelElement.style.fontWeight = 'bold';
      modelOut.appendChild(labelElement);
      div.appendChild(modelOut);
           
      video = document.createElement('video');
      video.style.display = 'block';
      video.width = div.clientWidth - 6;
      video.setAttribute('playsinline', '');
      video.onclick = () => { shutdown = true; };
      stream = await navigator.mediaDevices.getUserMedia(
          {video: { facingMode: "environment"}});
      div.appendChild(video);

      imgElement = document.createElement('img');
      imgElement.style.position = 'absolute';
      imgElement.style.zIndex = 1;
      imgElement.onclick = () => { shutdown = true; };
      div.appendChild(imgElement);
      
      const instruction = document.createElement('div');
      instruction.innerHTML = 
          '<span style="color: red; font-weight: bold;">' +
          'When finished, click here or on the video to stop this demo</span>';
      div.appendChild(instruction);
      instruction.onclick = () => { shutdown = true; };
      
      video.srcObject = stream;
      await video.play();

      captureCanvas = document.createElement('canvas');
      captureCanvas.width = 1280; //video.videoWidth;
      captureCanvas.height = 480; //video.videoHeight;
      window.requestAnimationFrame(onAnimationFrame);
      
      return stream;
    }
    async function stream_frame(label, imgData) {
      if (shutdown) {
        removeDom();
        shutdown = false;
        return '';
      }

      var preCreate = Date.now();
      stream = await createDom();
      
      var preShow = Date.now();
      if (label != "") {
        labelElement.innerHTML = label;
      }
            
      if (imgData != "") {
        var videoRect = video.getClientRects()[0];
        imgElement.style.top = videoRect.top + "px";
        imgElement.style.left = videoRect.left + "px";
        imgElement.style.width = videoRect.width + "px";
        imgElement.style.height = videoRect.height + "px";
        imgElement.src = imgData;
      }
      
      var preCapture = Date.now();
      var result = await new Promise(function(resolve, reject) {
        pendingResolve = resolve;
      });
      shutdown = false;
      
      return {'create': preShow - preCreate, 
              'show': preCapture - preShow, 
              'capture': Date.now() - preCapture,
              'img': result};
    }
    ''')

  display(js)
  
def video_frame(label, bbox):
  data = eval_js('stream_frame("{}", "{}")'.format(label, bbox))
  return data

In [62]:
# 参照画像の読み込み
ref = cv2.imread("poster_s.jpg")

# ORBを使う場合ORB_create　AKAZEならAKAZE_create
## BRIEFなら keypointがStarDetector_create (FAST特徴点)、descriptorがBriefDescriptorExtractor_create()
## SIFTは、特許関係の問題で、nonfreeオプションをONにしてビルドしないと入りません
## インストール済みの場合：cv2.xfeatures2d.SIFT_create() で使えます。Colabにはデフォルトでは入ってない

#orb = cv2.ORB_create()
orb = cv2.AKAZE_create()

# refに対する特徴点検出
kp_ref, des_ref = orb.detectAndCompute(ref, None)


In [64]:
# カメラからの画像読み込みと対応点描画

video_stream()
label_html = 'Capturing...'
img_disp = ''

img_a = np.ones([480,1280,1], dtype=np.uint8)*255
while True:
    js_reply = video_frame(label_html, img_disp)
    if not js_reply:
        break
    src = js_to_image(js_reply["img"])
    src = cv2.resize(src,(640,480))

    # 各画像に対するkeypointとdescriptorの計算
    kp_src, des_src = orb.detectAndCompute(src, None)

    # マッチング。ORBならHAMMING距離をつかうべき
    matcher = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
    matches = matcher.match(des_src,des_ref)

    # 全対応を可視化
    corr = cv2.drawMatches(src,kp_src,ref,kp_ref,matches,None,flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
    #cv2_imshow(corr_disp) # 表示

    img_disp = cv2.resize(corr,(1280,480))
    img_disp = np.append(img_disp, img_a, axis=2)
    img_disp = cv2.cvtColor(img_disp,cv2.COLOR_BGRA2RGBA)

    img_disp = img_to_bytes(img_disp)


<IPython.core.display.Javascript object>